In [1]:
import sqlite3
import time

In [3]:
# Connexion à la base de données SQLite
conn = sqlite3.connect('example_2.db')
cursor = conn.cursor()

# Création de la table
cursor.execute('''
CREATE TABLE IF NOT EXISTS clients (
    id INTEGER PRIMARY KEY,
    nom TEXT,
    age INTEGER,
    ville TEXT
)
''')

# Insertion de données de test
cursor.execute("INSERT INTO clients (nom, age, ville) VALUES ('Alice', 30, 'Paris');")

noms = ["Bob", "Charlie", "David", "Eve"] 
data = [(nom, age, ville) for nom in noms for age in range(20, 40, 5) for ville in ["Paris", "Lyon", "Marseille"]] * 20000 
cursor.executemany("INSERT INTO clients (nom, age, ville) VALUES (?, ?, ?)", data)

conn.commit()

In [4]:
# Fonction pour mesurer le temps de la requête sans index
def requete_sans_index():
    start_time = time.time()
    cursor.execute("SELECT * FROM clients WHERE nom = 'Alice'")
    results = cursor.fetchall()
    end_time = time.time()
    print(f"Temps de requête sans index: {end_time - start_time:.4f} secondes")
    print(f"Nombre de résultats: {len(results)}")

# Exécution de la fonction
requete_sans_index()

Temps de requête sans index: 5.2215 secondes
Nombre de résultats: 1


In [5]:
# Création de l'index
cursor.execute("CREATE INDEX IF NOT EXISTS idx_nom ON clients (nom)")

# Fonction pour mesurer le temps de la requête avec index
def requete_avec_index():
    start_time = time.time()
    cursor.execute("SELECT * FROM clients WHERE nom = 'Alice'")
    results = cursor.fetchall()
    end_time = time.time()
    print(f"Temps de requête avec index: {end_time - start_time:.4f} secondes")
    print(f"Nombre de résultats: {len(results)}")

# Exécution de la fonction
requete_avec_index()

conn.close()

Temps de requête avec index: 0.0111 secondes
Nombre de résultats: 1



1. **Requête sans index** :
   - **Temps de requête** : Plus long.
   - **Explication** : Sans index, la base de données doit effectuer un scan complet de la table pour trouver les enregistrements correspondants. Cela signifie qu'elle examine chaque ligne, ce qui est inefficace pour les grandes tables.

2. **Requête avec index** :
   - **Temps de requête** : Significativement plus court.
   - **Explication** : Avec un index sur la colonne `nom`, la base de données peut utiliser une structure de données optimisée (comme un arbre B+ dans beaucoup de systèmes de bases de données) pour localiser rapidement les enregistrements correspondants sans avoir à scanner toute la table.

#### Comparaison des résultats
- **Sans index** : Le temps de requête était beaucoup plus long car la base de données devait scanner toutes les lignes de la table `clients` pour trouver celles où `nom = 'Alice'`.
- **Avec index** : Le temps de requête a été considérablement réduit car l'index a permis d'accéder directement aux lignes correspondant à `nom = 'Alice'` sans un scan complet de la table.